# 一、课堂示例代码复刻

In [1]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [5]:
#导入数据
df = pd.read_csv(r'E:\[- -] C\文本数据分析\上课实例\text_analysis_weibo.csv', index_col = 0)
df

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东
...,...,...,...,...,...,...,...,...
49995,935天气路况早知道 2022年3月2日 星期三 今日限行尾号3和8 8：00分发布 ...,0,0,0,2231ab79796d569625633acafd9f1351,445,74,河南
49996,4月1日起，交通违法记分规则重大调整！ http://t.cn/A66zxfR0 ​​,0,0,0,91c9d380c1cc7811be5ee10ce510d77a,842,54,甘肃
49997,转发C,0,0,1,301771db13798e2dd493c02a44e59444,99,3433,江苏
49998,全体 甘肃车主注意 4月1日起，全面实施！ ...,0,0,0,NaN,0,0,甘肃


In [9]:
#数据太大，把数据换小一点
df = df.iloc[:200]
df

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,转发
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,刘雨昕运动者联濛河山覆冰雪健儿迎冬奥金牌全能唱跳不设限运动联濛开新年期待全能音乐人刘雨昕与运...
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,丁程鑫丁程鑫后援会官博丁程鑫超话丁程鑫二十成金筑梦鑫世界大年初一锁定北京卫视和门面主舞时代少...
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,诶你们真不要脸诶没资格宣传奥运抵制抵制
...,...,...,...,...,...,...,...,...,...
195,刘雨昕唱得真好听呀[虎爪比心],0,0,0,79995f74dc8c34bed446fc78f779349e,1215,370,云南,刘雨昕唱得真好听呀虎爪比心
196,转发C,0,0,0,e7d138414e5f6401d05a82ce1c10b21e,115,57,湖南,转发
197,转发C,0,0,0,3dc384fcc87182c886b4313d95c0efd1,580,141,上海,转发
198,//@一杯奶茶警告:二十四节气谁看了不说一句牛批,0,0,0,ead6274daba782bc90005aaffbe25bdd,78,836,广西,一杯奶茶警告二十四节气谁看了不说一句牛批


In [6]:
#语料预处理
#删除符号与数字
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)

'高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心'

In [7]:
# 分词，加载中文停用词词典
stopwords = open(r'E:\[- -] C\文本数据分析\上课实例\stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)
test = df['标题/微博内容'][0]
clean_text(test)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BITTER~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.181 seconds.
Prefix dict has been built successfully.


'高校 通报 教师 图书馆 打电话 声音 过大 出言不逊 公道 人心 谣言 人心 \u200b \u200b'

In [10]:
#处理原df中标题/微博内容中的内容，增添一列新的
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

C:\Users\bittersweet\AppData\Local\Temp\ipykernel_47196\2584316329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['标题/微博内容'] = df['标题/微博内容'].astype(str)
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_47196\2584316329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_47196\2584316329.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 公道, 人心, 谣..."
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,[转发]
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,"[刘雨昕, 运动, 濛, 河山, 覆, 冰雪, 健儿, 冬奥, 金牌, 全能, 唱, 跳, ..."
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,"[丁程鑫, 丁程鑫, 后援会, 官博丁, 程鑫, 超话, 丁程鑫, 二十, 成金筑梦鑫, 世..."
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,"[诶, 不要脸, 诶, 资格, 宣传, 奥运, 抵制, 抵制]"
...,...,...,...,...,...,...,...,...,...
195,刘雨昕唱得真好听呀[虎爪比心],0,0,0,79995f74dc8c34bed446fc78f779349e,1215,370,云南,"[刘雨昕, 唱, 好听, 虎爪, 比心]"
196,转发C,0,0,0,e7d138414e5f6401d05a82ce1c10b21e,115,57,湖南,[转发]
197,转发C,0,0,0,3dc384fcc87182c886b4313d95c0efd1,580,141,上海,[转发]
198,//@一杯奶茶警告:二十四节气谁看了不说一句牛批,0,0,0,ead6274daba782bc90005aaffbe25bdd,78,836,广西,"[一杯, 奶茶, 警告, 二十四节气, 一句, 牛批]"


## LDA

In [11]:
# 根据分词结果创建字典
dictionary = corpora.Dictionary(df['微博内容分词'])     
# 根据分词结果创建语料库
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     

In [19]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

In [20]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.216*"转发" + 0.015*"周深" + 0.007*"致敬" + 0.007*"青春" + 0.007*"圈"')
(1, '0.028*"冬奥" + 0.024*"加油" + 0.019*"冬奥会" + 0.019*"北京" + 0.015*"健儿"')
(2, '0.047*"运动" + 0.046*"刘雨昕" + 0.046*"冬奥" + 0.041*"濛" + 0.036*"北京"')
(3, '0.052*"未来" + 0.030*"唱响" + 0.026*"短片" + 0.022*"周深" + 0.021*"音乐"')
(4, '0.030*"运动" + 0.020*"刘雨昕" + 0.020*"濛" + 0.020*"冬奥" + 0.015*"送"')
(5, '0.039*"宋亚轩" + 0.036*"少年" + 0.024*"时代" + 0.023*"北京" + 0.022*"团"')
(6, '0.020*"冬奥" + 0.019*"冰雪" + 0.015*"未来" + 0.012*"哈哈哈" + 0.012*"烊"')
(7, '0.020*"乌拉" + 0.013*"少年" + 0.013*"喀秋莎" + 0.013*"人心" + 0.007*"时代"')
(8, '0.032*"九洲" + 0.032*"唐" + 0.028*"蔡" + 0.028*"徐坤" + 0.019*"燕京啤酒"')
(9, '0.023*"王源" + 0.012*"刘雨昕" + 0.012*"真的" + 0.012*"岁" + 0.012*"温柔"')


In [26]:
#可视化
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1)
# 显示可视化界面
pyLDAvis.display(lda_vis,local=True)

In [25]:
pyLDAvis.display?

# 二、语料库探索

In [27]:
import pandas as pd
try:
    m_df = pd.read_csv(r'E:\[- -] C\文本数据分析\搜狗全网新闻数据_utf-8_demo.csv', encoding='GBK')
    print("文件成功读取！")
except UnicodeDecodeError:
    print("GBK 编码无法解码该文件，尝试其他编码。")

文件成功读取！


In [28]:
m_df

,url,contenttitle,content
0,http://news.sohu.com/20120612/n345428229.shtml,公安机关销毁１０余万非法枪支 跨国武器走私渐起,中广网唐山６月１２日消息（记者汤一亮 庄胜春）据中国之声《新闻晚高峰》报道，今天（１２日）上...
1,http://news.sohu.com/20120607/n344998325.shtml,张绍刚发道歉信网友不认可：他的问题是俯视他人（图）,天津卫视求职节目《非你莫属》“晕倒门”事件余波未了，主持人张绍刚前日通过《非你莫属》节目组发...
2,http://news.sohu.com/20120604/n344745879.shtml,＃（关注夏收）（３）夫妻“麦客”忙麦收,临沂（山东），２０１２年６月４日 夫妻“麦客”忙麦收 ６月４日，在山东省临沂市郯城县郯城街道...
3,http://news.sohu.com/20120613/n345535702.shtml,欧洲杯大战在即 荷兰葡萄牙面临淘汰将背水一战,中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比...
4,http://news.sohu.com/20120601/n344598651.shtml,扎克伯格携妻罗马当街吃３０元麦当劳午餐（组图）,环球网记者李亮报道，正在意大利度蜜月的“脸谱”创始人扎克伯格与他华裔妻子的一举一动都处于媒体...
...,...,...,...
94,http://news.cn.yahoo.com/ypen/20120531/1082314...,联合国秘书长敦促叙利亚政府履行安南和平计划,中新网５月３１日电 据外电报道，联合国秘书长潘基文３１日敦促叙利亚政府尊重承诺，履行叙危机联...
95,http://news.cn.yahoo.com/ypen/20120531/1082438...,武汉一环卫工人因“占道”遭围殴 被泼滚烫米粉,荆楚网消息（记者张扬 通讯员殷莉红、刘永慧、刘妮）５月３１日，３名男子因嫌环卫车占道，竟将一...
96,http://news.cn.yahoo.com/newspic/news/23261/,女大学生夜行遇劫 遭割喉身亡（,NaN
97,http://news.cn.yahoo.com/newspic/news/17917/3/,俄总统竞选“梅普”或许换位,９月２４日，俄罗斯总统梅德韦杰夫在“统一俄罗斯党”代表大会上提议，由现任总理普京参加２０１２...


In [29]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = m_df['content'][0]
remove_nums(test)

'中广网唐山月日消息记者汤一亮庄胜春据中国之声新闻晚高峰报道今天日上午公安机关年缉枪制爆专项行动统一销毁非法枪爆物品活动在河北唐山正式启动万余只非法枪支余吨炸药在全国个城市被统一销毁泼鳎合衷谖倚布全国缉枪制爆统一销毁行动开始孀殴安部副部长黄明一声令下大量仿制式枪以及猎枪火药枪气枪在河北唐山钢铁厂被投入炼钢炉与此同时在全国各省区市个城市破案追缴和群众主动上缴的万余支非法枪支被集中销毁在全国各指定场所余吨炸药被分别销毁公安部治安局局长刘绍武介绍这次销毁的非法枪支来源于三个方面跎芪洌捍蚧髌瓢赴括涉黑涉恶的团伙犯罪毒品犯罪还有从境外非法走私的枪支爆炸物谙毁现场记者看到了被追缴和上缴的各式各样的枪支跎芪洌阂舶括制式枪有的是军用枪仿制的制式抢还有猎枪私制的火药枪等等按照我国的枪支管理法这些都是严厉禁止个人非法持有的中国是世界上持枪犯罪的犯罪率最低的国家之一忻懒手破获特大跨国走私武器弹药案日中美执法部门联手成功破获特大跨国走私武器弹药案在中国抓获犯罪嫌疑人名缴获各类枪支支子弹万余发及大量枪支配件在美国抓获犯罪嫌疑人名缴获各类枪支支这是公安部与美国移民海关执法局通过联合调查方式侦破重大跨国案件的又一成功案例玻埃保蹦辏冈拢玻等眨上海浦东国际机场海关在对美国纽约发往浙江台州申报品名为扩音器音箱的快件进行查验时发现货物内藏有手枪支枪支配件件长枪部件件经检验这些都是具有杀伤力的制式枪支及其配件这引起了公安部和海关总署的高度重视安部刑侦局局长刘安成因为是从海关进口的货物中检查出来夹带说明来源地是境外或是说国外这应该是一起特大跨国走私武器弹药的案件虾泄安局和上海海关缉私局成立联合专案组迅速开展案件侦查专案组于月日在浙江台州取件处将犯罪嫌疑人王挺男岁台州市人抓获王挺交代他通过一境外网站上认识了上家林志富年月以来林志富长期居住美国他通过互联网组建了一个走私贩卖私藏枪支弹药的群体通过网络在国内寻找枪支弹药买家并通过美国联邦速递公司将枪支弹药从纽约快递给多名类似王挺的中间人再通过中间人发送给国内买家税钢校犯罪分子依托虚拟网络进行犯罪交易隐蔽性强涉案人员使用的身份地址联系方式都是虚构的侦查难度很大刘安成说此案体现了是新型犯罪特别是现代犯罪的新特点醢渤桑核不受距离的限制经常是跨国跨境甚至是跨一个数个甚至数十个国家这种犯罪手法的改变和新型犯罪的特点要求我们各国警方充分合作髡撸禾酪涣痢胜春'

In [30]:
# 加载中文停用词词典，可个性化设置
stopwords = open(r'E:\[- -] C\文本数据分析\上课实例\stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)
test = m_df['content'][0]
clean_text(test)

'中广网 唐山 ６ 月 １ ２ 日 消息 汤一亮 \u3000 庄胜春 中国 之声 新闻 晚 高峰 报道 １ ２ 日 上午 公安机关 ２ ０ １ ２ 年缉枪 制爆 专项 统一 销毁 非法 枪爆 物品 活动 河北 唐山 正式 启动 １ ０ 万余只 非法 枪支 ２ ５ ０ 余吨 炸药 全国 １ ５ ０ 城市 统一 销毁 泼 鳎 合衷 谖 倚布 全国 缉枪 制爆 统一 销毁 Ｋ 孀 殴安 部副 部长 黄明 一声令下 仿制 式 枪 猎枪 火药枪 气枪 河北 唐山 钢铁厂 投入 炼钢炉 全国 省区市 １ ５ ０ 城市 破案 追缴 群众 主动 上缴 １ ０ 万余支 非法 枪支 销毁 全国 指定 场所 ２ ５ ０ 余吨 炸药 销毁 公安部 治安 局局长 刘 绍武 介绍 这次 销毁 非法 枪支 来源于 三个 Ａ 跎 芪 洌 捍 蚧 髌 瓢 赴括 涉黑 涉恶 团伙 犯罪 毒品 犯罪 境外 非法 走私 枪支 爆炸物 Ｔ 谙毁 现场 追缴 上缴 各式各样 枪支 Ａ 跎 芪 洌 阂 舶括 制式 枪 有的是 军用 枪 仿制 制式 抢 猎枪 私制 火药枪 我国 枪支 管理法 严厉 禁止 非法 持有 中国 世界 持枪 犯罪 犯罪率 最低 国家 Ｖ 忻懒手 破获 特大 跨国 走私 武器弹药 案 ＝ 日 美 执法 部门 联手 成功 破获 特大 跨国 走私 武器弹药 案 中国 抓获 犯罪 嫌疑人 ２ ３ 名 缴获 各类 枪支 ９ ３ 支 子弹 ５ 万余 发及 枪支 配件 美国 抓获 犯罪 嫌疑人 ３ 名 缴获 各类 枪支 １ ２ 支 这是 公安部 美国移民 海关 执法局 联合 调查 方式 侦破 跨国 案件 成功 案例 ＃ 玻埃保 蹦 辏 冈 拢 玻 眨 上海浦东国际机场 海关 美国纽约 发 浙江 台州 申报 品名 扩音器 音箱 快件 查验 发现 货物 藏有 手枪 ９ 支 枪支 配件 ９ 长枪 部件 ７ 检验 杀伤力 制式 枪支 配件 公安部 海关总署 高度重视 ９ 安部 刑侦局 局长 刘安 是从 海关 进口 货物 检查 夹带 来源 地是 境外 国外 特大 跨国 走私 武器弹药 案件 Ｉ 虾 Ｊ 泄安局 上海海关 缉私局 成立 联合 专案组 迅速开展 案件 侦查 专案组 ８ 月 ２ ６ 日 浙江 台州 Ｕ Ｐ Ｓ 取件 犯罪 嫌疑人 王挺 ３ ２ 岁 台州市 抓获 王挺 交代 境外 网站 上家 林志富 

In [31]:
m_df['content'] = m_df['content'].astype(str)
m_df['content_new'] = m_df['content'].apply(remove_nums)
m_df['content_new'] = m_df['content_new'].apply(clean_text)
m_df['content_new'] = m_df['content_new'].apply(lambda x: x.split())
m_df

,url,contenttitle,content,content_new
0,http://news.sohu.com/20120612/n345428229.shtml,公安机关销毁１０余万非法枪支 跨国武器走私渐起,中广网唐山６月１２日消息（记者汤一亮 庄胜春）据中国之声《新闻晚高峰》报道，今天（１２日）上...,"[中广网, 唐山, 月, 日, 消息, 汤一亮, 庄胜春据, 中国, 之声, 新闻, 晚, ..."
1,http://news.sohu.com/20120607/n344998325.shtml,张绍刚发道歉信网友不认可：他的问题是俯视他人（图）,天津卫视求职节目《非你莫属》“晕倒门”事件余波未了，主持人张绍刚前日通过《非你莫属》节目组发...,"[天津, 卫视, 求职, 节目, 非你莫属, 晕倒, 事件, 余波, 未了, 主持人, 张绍..."
2,http://news.sohu.com/20120604/n344745879.shtml,＃（关注夏收）（３）夫妻“麦客”忙麦收,临沂（山东），２０１２年６月４日 夫妻“麦客”忙麦收 ６月４日，在山东省临沂市郯城县郯城街道...,"[临沂, 山东, 年月日, 夫妻, 麦客, 忙, 麦收, 月, 日, 山东省, 临沂市, 郯..."
3,http://news.sohu.com/20120613/n345535702.shtml,欧洲杯大战在即 荷兰葡萄牙面临淘汰将背水一战,中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比...,"[中广网, 北京, 月, 日, 消息, 王宇据, 中国, 之声, 新闻, 晚, 高峰, 报道..."
4,http://news.sohu.com/20120601/n344598651.shtml,扎克伯格携妻罗马当街吃３０元麦当劳午餐（组图）,环球网记者李亮报道，正在意大利度蜜月的“脸谱”创始人扎克伯格与他华裔妻子的一举一动都处于媒体...,"[环球网, 李亮, 报道, 意大利, 度蜜月, 脸谱, 创始人, 扎克, 伯格, 华裔, 妻..."
...,...,...,...,...
94,http://news.cn.yahoo.com/ypen/20120531/1082314...,联合国秘书长敦促叙利亚政府履行安南和平计划,中新网５月３１日电 据外电报道，联合国秘书长潘基文３１日敦促叙利亚政府尊重承诺，履行叙危机联...,"[中新网, 月, 日电, 外电报道, 联合国, 秘书长, 潘基文, 日, 敦促, 叙利亚, ..."
95,http://news.cn.yahoo.com/ypen/20120531/1082438...,武汉一环卫工人因“占道”遭围殴 被泼滚烫米粉,荆楚网消息（记者张扬 通讯员殷莉红、刘永慧、刘妮）５月３１日，３名男子因嫌环卫车占道，竟将一...,"[荆楚网, 消息, 张扬, 通讯员, 殷莉红, 刘永慧, 刘妮, 月, 日名, 男子, 因嫌..."
96,http://news.cn.yahoo.com/newspic/news/23261/,女大学生夜行遇劫 遭割喉身亡（,nan,[]
97,http://news.cn.yahoo.com/newspic/news/17917/3/,俄总统竞选“梅普”或许换位,９月２４日，俄罗斯总统梅德韦杰夫在“统一俄罗斯党”代表大会上提议，由现任总理普京参加２０１２...,"[月, 日, 俄罗斯, 总统, 梅德韦, 杰夫, 统一, 俄罗斯, 党代表大会, 提议, 现..."


In [32]:
dictionary = corpora.Dictionary(m_df['content_new'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in m_df['content_new']]     # 根据分词结果创建语料库

In [33]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

In [34]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.010*"投资" + 0.010*"资产" + 0.009*"银行" + 0.008*"贷款" + 0.007*"政府"')
(1, '0.008*"山西" + 0.008*"中国" + 0.006*"大豆" + 0.005*"月" + 0.005*"日"')
(2, '0.016*"食品安全" + 0.013*"臁" + 0.011*"量化" + 0.010*"服务" + 0.009*"餐饮"')
(3, '0.010*"电价" + 0.010*"枪支" + 0.007*"中国" + 0.007*"犯罪" + 0.007*"需求"')
(4, '0.009*"庞飒" + 0.008*"银行" + 0.007*"国债" + 0.007*"增持" + 0.006*"基金"')
(5, '0.013*"存款" + 0.010*"银行" + 0.010*"企业家" + 0.006*"经济" + 0.005*"制度"')
(6, '0.010*"生产" + 0.006*"单位" + 0.006*"月" + 0.006*"油价" + 0.005*"李旭利"')
(7, '0.012*"利率" + 0.007*"存款" + 0.007*"月" + 0.007*"银行" + 0.006*"高尔夫球场"')
(8, '0.014*"信托" + 0.012*"退税" + 0.010*"投资" + 0.009*"艺术品" + 0.009*"出口"')
(9, '0.013*"文化" + 0.011*"大连" + 0.009*"新区" + 0.008*"投资" + 0.007*"建设"')


In [35]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1)
# 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

In [37]:
#检查每个文档的主题分布，查看哪些文档更多地涉及到“犯罪”相关的主题。

# 导入必要的库
from gensim import corpora, models

# 定义一个函数来分析每个文档中与“犯罪”相关的主题所占比例
def analyze_document_topic_distribution(lda_model, corpus, keyword='犯罪'):
    """
    分析每个文档中与指定关键词相关的主题所占比例
    
    参数：
    lda_model: 训练好的 LDA 模型
    corpus: 文档的词袋表示的语料库
    keyword: 指定关键词，默认为“犯罪”
    
    返回值：
    crime_topic_documents: 与“犯罪”相关的文档列表
    """
    # 初始化与“犯罪”相关的文档列表
    crime_topic_documents = []
    
    # 获取每个文档的主题分布
    document_topic_distributions = get_document_topic_distribution(lda_model, corpus)
    
    # 遍历每个文档的主题分布
    for idx, topic_distribution in enumerate(document_topic_distributions):
        # 检查每个主题
        for topic, prob in topic_distribution:
            # 获取主题的词汇分布
            topic_words = lda_model.show_topic(topic)
            # 检查主题中是否有包含关键词的词汇
            for word, _ in topic_words:
                if word.startswith(keyword):
                    # 如果找到包含关键词的词汇，将该文档添加到列表中
                    crime_topic_documents.append(idx)
                    break  # 停止查找其他词汇，因为已经发现与“犯罪”相关的主题
            else:
                continue  # 如果没有找到包含关键词的词汇，继续查找下一个主题
            break  # 如果找到包含关键词的词汇，停止查找其他主题
    
    return crime_topic_documents

# 使用示例
crime_documents = analyze_document_topic_distribution(lda_model, corpus)

# 输出结果
print("与犯罪相关的文档：", crime_documents)


与犯罪相关的文档： [0, 4, 11, 14, 20, 24, 29, 30, 33, 36, 38, 40, 41, 45, 48, 49, 59, 60, 68, 79, 86, 88, 93, 96, 98]


100行里面有25行新闻都与“犯罪”主题相关。

In [42]:
#词汇共现分析：
#对文本数据进行词汇共现分析，找出与“犯罪”一起出现频率较高的词汇。可以构建词汇共现矩阵，然后计算每个词汇与“犯罪”同时出现的频率，从而找出最相关的词汇。

from gensim.models.coherencemodel import CoherenceModel

# 提供文本数据作为参数
texts = m_df['content_new'].tolist()

# 构建词汇共现矩阵
coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')

# 获取词汇共现矩阵
coherence_matrix = coherence_model.get_coherence_per_topic()

# 输出一些调试信息
print("词汇共现矩阵形状：", len(coherence_matrix))
print("字典中“犯罪”这个词汇的索引：", dictionary.token2id.get('犯罪'))

# 获取词汇共现矩阵中与“犯罪”相关的词汇频率
if '犯罪' not in dictionary.token2id:
    print("词汇表中没有包含“犯罪”这个词汇。")
else:
    crime_index = dictionary.token2id.get('犯罪')
    if crime_index >= len(coherence_matrix):
        print("索引超出了范围。")
    else:
        crime_related_words_frequency = coherence_matrix[crime_index]
        print("与犯罪一起出现频率较高的词汇：", crime_related_words_frequency)



词汇共现矩阵形状： 10
字典中“犯罪”这个词汇的索引： 160
索引超出了范围。



在输出中，我们首先看到词汇共现矩阵的形状为 10，这意味着共现矩阵中有 10 行。然后，我们发现字典中“犯罪”这个词汇的索引为 160，这表明在字典中，“犯罪”这个词汇的索引超出了共现矩阵的范围。

这可能是由于字典中的某些词汇在共现矩阵中并没有出现，或者共现矩阵的大小与预期不符。为了进一步解决这个问题，我们需要检查一下训练 LDA 模型时使用的语料库和字典是否正确。可能需要重新构建语料库和字典，确保其中包含了所有需要的词汇。

怎么对“犯罪”进行进一步分析呢?

没有出现犯罪的主题，有犯罪的文档

下一步去找与“犯罪”相关的主题，相关的词汇？